In [ ]:
pip install langgraph openai requests

In [ ]:
pip install --upgrade --force-reinstall langgraph

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경변수 로드

True

#### Tool

In [15]:
from langchain_teddynote.tools.tavily import TavilySearch

In [16]:
tool = TavilySearch(max_results=5)

tools = [tool]

In [17]:
# 도구 실행
print(tool.invoke("LangGraph Tutorial"))

[{'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?', 'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial', 'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactions.', 'score': 0.91137725, 'raw_content': 'Published Time: 2024-06-26T21:00:00.000Z\nLangGraph Tutorial: What Is

#### LLM + Tools

In [168]:
from typing import TypedDict, List, Optional, Dict

class StartupCandidate(TypedDict):
    name: str
    summary: Optional[str]

class MarketScores(TypedDict):
    market_size: int
    problem_fit: int
    willingness_to_pay: int
    revenue_model_clarity: int
    upside_potential: int

class AgentState(TypedDict):
    user_query: str
    domain: str
    startup_candidates: List[StartupCandidate]
    current_index: int
    candidates_documents: List[StartupCandidate]
    evaluation_scores: Dict[str, MarketScores]
    evaluation_summary: Dict[str, str]

In [169]:
# 입력 State 예시
input = {
    'user_query': 'AI 헬스케어 스타트업 알려줘',
    'startup_candidates': [
        {
            'name': 'Medibot',
            'summary': 'Medibot은 AI 기반 진단 시스템으로 빠르고 정확한 진료 보조를 제공. AI 알고리즘을 통해 의료 데이터 분석을 지원.'
        }
        ,{
            'name':'Medibot22',
            'summary':'Medibot은 AI 기반 진단 시스템으로 빠르고 정확한 진료 보조를 제공. AI 알고리즘을 통해 의료 데이터 분석을 지원.'
        }
    ],
    'domain': '헬스케어'
}

In [170]:
from langchain_teddynote.tools.tavily import TavilySearch

def fetch_tavily_context(startup_name: str) -> str:
    search = TavilySearch()
    query = f"{startup_name} 시장성 OR 성장 가능성 OR 투자 현황 OR 경쟁사 비교"
    
    try:
        results = search.run(query)
        if not results:
            return f"{startup_name} 관련 정보가 충분하지 않습니다."
        return "\n".join(results[:3])[:3000]
    except Exception as e:
        return f"Tavily 검색 중 오류 발생: {str(e)}"

def check_context_node(state: AgentState) -> AgentState:
    current = state["startup_candidates"][state["current_index"]]
    summary = current.get("summary")

    if not summary or len(summary.strip()) < 10:
        summary = fetch_tavily_context(current["name"])
        current["summary"] = summary

    state["startup_candidates"][state["current_index"]] = current
    return state

#### 시장성 평가

In [171]:
def extract_json_from_response(response: str) -> str:
    try:
        if "```json" in response:
            return response.split("```json")[1].split("```")[0].strip()
        match = re.search(r"({[\\s\\S]*})", response)
        if match:
            return match.group(1)
    except Exception as e:
        print(f"[ERROR] JSON 추출 실패: {e}")
    return ""

In [172]:
def generate_market_evaluation(startup_name: str, summary: str, domain: str, context: str = "") -> Tuple[Dict[str, int], str]:
    prompt = f"""
아래는 스타트업에 대한 개요입니다:

[회사명]: {startup_name}
[개요]: {summary}

이 스타트업의 **시장성**을 다음 항목에 맞춰 분석해 주세요:

- 이 시장은 얼마나 큰가? (market_size)
- 제품이 시장의 실제 문제를 해결하는가? (problem_fit)
- 고객이 실제로 이 제품에 비용을 지불할 이유가 있는가? (willingness_to_pay)
- 수익 모델은 명확한가? (revenue_model_clarity)
- 이 스타트업이 성공한다면, 정말 큰 기회가 될까? (upside_potential)

각 항목에 대해 0~10점으로 평가하고, 요약도 함께 작성해 주세요.

반드시 아래 JSON 형식으로 출력하세요:
```json
{{
 "evaluation_scores": {{
    "market_size": 0,
    "problem_fit": 0,
    "willingness_to_pay": 0,
    "revenue_model_clarity": 0,
    "upside_potential": 0
  }},
  "evaluation_summary": "..."
}}
"""
    response = llm.invoke(prompt).content
    print("[DEBUG] LLM 응답 원본:\n", response)

    try:
        json_str = extract_json_from_response(response)
        print("[DEBUG] 추출된 JSON:\n", json_str)
        parsed = json.loads(json_str)

        # 강제 검증
        scores = parsed.get("evaluation_scores", {})
        summary = parsed.get("evaluation_summary", "")

        if not isinstance(scores, dict):
            raise ValueError("evaluation_scores is not a dictionary")

        expected_keys = [
            "market_size", "problem_fit", "willingness_to_pay",
            "revenue_model_clarity", "upside_potential"
        ]
        for key in expected_keys:
            value = scores.get(key)
            if not isinstance(value, int):
                raise ValueError(f"[{key}] is not int: {value} (type={type(value)})")

        return scores, summary

    except Exception as e:
        print(f"[ERROR] JSON parsing failed: {e}")
        return {
            "market_size": 5,
            "problem_fit": 5,
            "willingness_to_pay": 5,
            "revenue_model_clarity": 5,
            "upside_potential": 5
        }, "⚠ JSON 파싱 실패로 기본 점수를 반환합니다."

In [173]:
def check_context_node(state: AgentState) -> AgentState:
    current = state["startup_candidates"][state["current_index"]]
    summary = current.get("summary", "")
    if not summary or len(summary.strip()) < 10:
        current["summary"] = fetch_tavily_context(current["name"])
    state["startup_candidates"][state["current_index"]] = current
    return state

In [174]:
def analyze_market_node(state: AgentState) -> AgentState:
    candidate = state["startup_candidates"][state["current_index"]]
    name = candidate["name"]
    print(f"[INFO] 분석 중: {name}")

    summary = candidate["summary"]
    domain = state["domain"]
    context = summary

    scores, summary_text = generate_market_evaluation(
            startup_name=name,
            summary=summary,
            domain=state["domain"],
            context=summary
        )

    state["candidates_documents"].append(candidate)
    state["evaluation_scores"][name] = scores
    state["evaluation_summary"][name] = summary_text

    state["current_index"] += 1  # ✅ 실제 상태 변경은 여기서
    return state

#### 최종정리

In [175]:
def aggregate_results_node(state: AgentState) -> AgentState:
    return {
    "candidates_documents": state["candidates_documents"],
    "domain": state["domain"],
    "evaluation_scores": state["evaluation_scores"],
    "evaluation_summary": state["evaluation_summary"]  # ✅ 그대로 dictionary로 유지
}

#### Langgraph 구성

In [176]:
def analyze_all_and_finish(state: AgentState) -> AgentState:
    for candidate in state["startup_candidates"]:
        name = candidate["name"]
        summary = candidate.get("summary", "")
        
        # 🔍 summary 부족 시 Tavily로 보완
        if not summary or len(summary.strip()) < 10:
            summary = fetch_tavily_context(name)
        candidate["summary"] = summary

        # 💡 시장성 평가 수행
        scores, summary_text = generate_market_evaluation(
            startup_name=name,
            summary=summary,
            domain=state["domain"],
            context=summary
        )

        # 📌 결과 누적
        state["candidates_documents"].append(candidate)
        state["evaluation_scores"][name] = scores
        state["evaluation_summary"][name] = summary_text

        print(f"[INFO] ✅ 분석 완료: {name}")

    return {
        "candidates_documents": state["candidates_documents"],
        "domain": state["domain"],
        "evaluation_scores": state["evaluation_scores"],
        "evaluation_summary": state["evaluation_summary"]  # ✅ 그대로 dictionary로 유지
    }

In [177]:
from langgraph.graph import StateGraph, END
graph = StateGraph(AgentState)

graph = StateGraph(AgentState)
graph.add_node("analyze_all_and_finish", analyze_all_and_finish)
graph.set_entry_point("analyze_all_and_finish")
graph.add_edge("analyze_all_and_finish", END)

market_analysis_app = graph.compile()


In [179]:
input_state = {
    "user_query": "AI 헬스케어 스타트업 알려줘",
    "domain": "헬스케어",
    "startup_candidates": [
        {"name": "Medibot", "summary": "Medibot은 AI 기반 진단 시스템으로 빠르고 정확한 진료 보조를 제공."},
        {"name": "닥터나우", "summary": "완전 개꿀이야 닥터나우"}
    ],
    "current_index": 0,
    "candidates_documents": [],
    "evaluation_scores": {},
    "evaluation_summary": {}
}

result = market_analysis_app.invoke(input_state)
print(result)

[DEBUG] LLM 응답 원본:
 
```json
{
 "evaluation_scores": {
    "market_size": 8,
    "problem_fit": 9,
    "willingness_to_pay": 9,
    "revenue_model_clarity": 8,
    "upside_potential": 9
  },
  "evaluation_summary": "Medibot은 건강 관리 분야에서의 AI 기반 진단 시스템으로 매우 큰 시장 규모를 대상으로 하고 있습니다. 제품은 실제 진료 과정에서 발생하는 문제를 해결하며, 고객들은 빠르고 정확한 진료 보조에 대해 비용을 지불할 의향이 있습니다. 수익 모델도 명확하고, 성공한다면 큰 기회를 가질 수 있는 스타트업으로 평가됩니다."
}
```
[DEBUG] 추출된 JSON:
 {
 "evaluation_scores": {
    "market_size": 8,
    "problem_fit": 9,
    "willingness_to_pay": 9,
    "revenue_model_clarity": 8,
    "upside_potential": 9
  },
  "evaluation_summary": "Medibot은 건강 관리 분야에서의 AI 기반 진단 시스템으로 매우 큰 시장 규모를 대상으로 하고 있습니다. 제품은 실제 진료 과정에서 발생하는 문제를 해결하며, 고객들은 빠르고 정확한 진료 보조에 대해 비용을 지불할 의향이 있습니다. 수익 모델도 명확하고, 성공한다면 큰 기회를 가질 수 있는 스타트업으로 평가됩니다."
}
[INFO] ✅ 분석 완료: Medibot
[DEBUG] LLM 응답 원본:
 
```json
{
 "evaluation_scores": {
    "market_size": 8,
    "problem_fit": 7,
    "willingness_to_pay": 6,
    "revenue_model_clarity": 5,
    "upside_potential": 

In [101]:
market_analysis_app.get_graph().draw_mermaid()

'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\tanalyze_all_and_finish(analyze_all_and_finish)\n\t__end__([<p>__end__</p>]):::last\n\t__start__ --> analyze_all_and_finish;\n\tanalyze_all_and_finish --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'